In [59]:
import hcl2

In [60]:
vmTemplateFile = "runner-images/images/windows/templates/windows-2019.pkr.hcl"
outFile = "runner-images/images/windows/templates/output.pkr.hcl"

In [61]:
with open(vmTemplateFile, 'r') as f:
    vmTemplateHcl = hcl2.parse(f)

In [62]:
with open(outFile, 'w') as f:
    f.write(hcl2.writes(vmTemplateHcl))

In [63]:
vmTemplateHcl

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'body'), [Tree(Token('RULE', 'block'), [Tree(Token('RULE', 'identifier'), [Token('NAME', 'locals')]), Tree(Token('RULE', 'body'), [Tree(Token('RULE', 'new_line_or_comment'), [Token('NL_OR_COMMENT', '\n  ')]), Tree(Token('RULE', 'attribute'), [Tree(Token('RULE', 'identifier'), [Token('NAME', 'managed_image_name')]), Token('EQ', ' ='), Tree(Token('RULE', 'binary_op'), [Tree(Token('RULE', 'expr_term'), [Tree(Token('RULE', 'get_attr_expr_term'), [Tree(Token('RULE', 'expr_term'), [Tree(Token('RULE', 'identifier'), [Token('NAME', 'var')])]), Tree(Token('RULE', 'get_attr'), [Tree(Token('RULE', 'identifier'), [Token('NAME', 'managed_image_name')])])])]), Tree(Token('RULE', 'binary_term'), [Tree(Token('RULE', 'binary_operator'), [Token('BINARY_OP', '!=')]), Tree(Token('RULE', 'conditional'), [Tree(Token('RULE', 'expr_term'), [Token('STRING_LIT', '""')]), Tree(Token('RULE', 'expr_term'), [Tree(Token('RULE', 'get_attr_expr_term'), [Tree(Token('RULE

In [64]:
from copy import deepcopy
from lark import Token, Tree

In [65]:
def build_tags_tree(base_indent: int = 0) -> Tree:
    # build Tree representing following HCL2 structure
    # tags = {
    #   Name = "My bucket"
    #   Environment = "Dev"
    # }
    return Tree('attribute', [
        Tree('identifier', [
            Token('NAME', 'tags')
        ]),
        Token('EQ', '='),
        Tree('expr_term', [
            Tree('object', [
                Tree('new_line_or_comment', [
                    Token('NL_OR_COMMENT', '\n' + '  ' * (base_indent + 1)),
                ]),
                Tree('object_elem', [
                    Tree('identifier', [
                        Token('NAME', 'Name')
                    ]),
                    Token('EQ', '='),
                    Tree('expr_term', [
                        Token('STRING_LIT', '"My bucket"')
                    ])
                ]),
                Tree('new_line_and_or_comma', [
                    Tree('new_line_or_comment', [
                        Token('NL_OR_COMMENT', '\n' + '  ' * (base_indent + 1)),
                    ]),
                ]),
                Tree('object_elem', [
                    Tree('identifier', [
                        Token('NAME', 'Environment')
                    ]),
                    Token('EQ', '='),
                    Tree('expr_term', [
                        Token('STRING_LIT', '"Dev"')
                    ])
                ]),
                Tree('new_line_and_or_comma', [
                    Tree('new_line_or_comment', [
                        Token('NL_OR_COMMENT', '\n' + '  ' * base_indent),
                    ]),
                ]),
            ]),
        ])
    ])


def is_bucket_block(tree: Tree) -> bool:
    # check whether given Tree represents `resource "aws_s3_bucket" "bucket"`
    try:
        return tree.data == 'block' and tree.children[2].value == '"bucket"'
    except (IndexError, AttributeError):
        return False


def insert_tags(tree: Tree, indent: int = 0) -> Tree:
    # Insert tags tree and adjust surrounding whitespaces to match indentation
    new_children = [*tree.children.copy(), build_tags_tree(indent)]
    # add indentation before tags tree
    new_children[len(tree.children) - 1] = Tree('new_line_or_comment', [
        Token('NL_OR_COMMENT', '\n  ')
    ])
    # move closing bracket to the new line
    new_children.append(
        Tree('new_line_or_comment', [
            Token('NL_OR_COMMENT', '\n')
        ])
    )
    return Tree(tree.data, new_children)


def process_token(node: Token, indent=0):
    # Print details of this token and return its copy
    print(f'[{indent}] (token)\t|', ' ' * indent, node.type, node.value)
    return deepcopy(node)


def process_tree(node: Tree, depth=0) -> Tree:
    # Recursively iterate over tree's children
    # the depth parameter represents recursion depth,
    #   it's used to deduce indentation for printing tree and for adjusting whitespace after adding tags
    new_children = []
    print(f'[{depth}] (tree)\t|', ' ' * depth, node.data)
    for child in node.children:
        if isinstance(child, Tree):
            if is_bucket_block(child):
                block_children = child.children.copy()
                # this child is the Tree representing block's actual body
                block_children[3] = insert_tags(block_children[3], depth)
                # replace original Tree with new one including the modified body
                child = Tree(child.data, block_children)

            new_children.append(process_tree(child, depth + 1))

        else:
            new_children.append(process_token(child, depth + 1))

    return Tree(node.data, new_children)

In [67]:
    tree = vmTemplateHcl
    new_tree = process_tree(tree)
    reconstructed = hcl2.writes(new_tree)

    with open(outFile, 'w') as f:
        f.write(reconstructed)

[0] (tree)	|  start
[1] (tree)	|   body
[2] (tree)	|    block
[3] (tree)	|     identifier
[4] (token)	|      NAME locals
[3] (tree)	|     body
[4] (tree)	|      new_line_or_comment
[5] (token)	|       NL_OR_COMMENT 
  
[4] (tree)	|      attribute
[5] (tree)	|       identifier
[6] (token)	|        NAME managed_image_name
[5] (token)	|       EQ  =
[5] (tree)	|       binary_op
[6] (tree)	|        expr_term
[7] (tree)	|         get_attr_expr_term
[8] (tree)	|          expr_term
[9] (tree)	|           identifier
[10] (token)	|            NAME var
[8] (tree)	|          get_attr
[9] (tree)	|           identifier
[10] (token)	|            NAME managed_image_name
[6] (tree)	|        binary_term
[7] (tree)	|         binary_operator
[8] (token)	|          BINARY_OP !=
[7] (tree)	|         conditional
[8] (tree)	|          expr_term
[9] (token)	|           STRING_LIT ""
[8] (tree)	|          expr_term
[9] (tree)	|           get_attr_expr_term
[10] (tree)	|            expr_term
[11] (tree)	|       